In [48]:
import pandas as pd
import bs4 as bs
from bs4 import SoupStrainer
import requests
import random
from bs4 import BeautifulSoup
import urllib.request, urllib.error, urllib.parse
import time
import numpy as np
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed

In [50]:
# function to get page html
def request_page(url):

    HEADERS = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Cache-Control": "max-age=0",
    }

    r = requests.get(url, headers=HEADERS)
    
    return r.text

In [52]:
# Initialize lists to store the results
weblinks = []
filing_dates = []
company_names = []
filing_types = []

In [54]:
# Function to fetch the web links, filing dates, company name, and filing type for a single company
def fetch_filing_data(company_name, file_type):
    company_weblinks = []
    company_filing_dates = []
    company_filing_types = []
    company_list = []
    
    # Construct the link for each company
    link = f'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&company={company_name.replace(" ", "+")}&type={file_type}&dateb=&owner=exclude&count=1000'

    try:
        # Fetch the page content
        html = request_page(link)
        soup = BeautifulSoup(html, "html.parser")

        # Find the table containing the filings
        table = soup.find('table', class_='tableFile2')
        
        # Extract document links, filing dates, and filing types if the table exists
        if table:
            rows = table.find_all('tr')[1:]  # Skip the header row
            for row in rows:
                cols = row.find_all('td')

                # Ensure there are enough columns
                if len(cols) >= 4:
                    # Extract document link
                    document_link_tag = cols[1].find('a')
                    if document_link_tag and 'href' in document_link_tag.attrs:
                        document_link = 'https://www.sec.gov' + document_link_tag['href'].strip()

                        # Extract filing date
                        filing_date = cols[3].text.strip()

                        # Extract filing type
                        filing_type = cols[0].text.strip()

                        # Filter out filings before 2010
                        filing_year = int(filing_date.split('-')[0])
                        if filing_year >= 2010:
                            company_weblinks.append(document_link)
                            company_filing_dates.append(filing_date)
                            company_filing_types.append(filing_type)
                            company_list.append(company_name)  # Append the company name for each row
        
        # Add a delay to avoid being blocked
        time.sleep(np.random.uniform(2, 5))

    except Exception as e:
        print(f"Error fetching data for {company_name}: {e}")

    return company_weblinks, company_filing_dates, company_filing_types, company_list



### Function call to get links for 10-K reports

In [57]:
fn_return_10k = fetch_filing_data('gap inc','10-K')
# Create a DataFrame from the results
df_final_10k = pd.DataFrame({
    "Company Name": fn_return_10k[3],
    "Filing Type": fn_return_10k[2],
    "Filing Date": fn_return_10k[1],
    "Weblink": fn_return_10k[0]
})

### Function call to get links for 10-Q reports

In [59]:
fn_return_10q = fetch_filing_data('gap inc','10-Q')
# Create a DataFrame from the results
df_final_10q = pd.DataFrame({
    "Company Name": fn_return_10q[3],
    "Filing Type": fn_return_10q[2],
    "Filing Date": fn_return_10q[1],
    "Weblink": fn_return_10q[0]
})

In [60]:
df_final = pd.concat([df_final_10q, df_final_10k])

In [61]:
df_final

,Company Name,Filing Type,Filing Date,Weblink
0,gap inc,10-Q,2024-08-30,https://www.sec.gov/Archives/edgar/data/39911/...
1,gap inc,10-Q,2024-05-31,https://www.sec.gov/Archives/edgar/data/39911/...
2,gap inc,10-Q,2023-11-21,https://www.sec.gov/Archives/edgar/data/39911/...
3,gap inc,10-Q,2023-08-25,https://www.sec.gov/Archives/edgar/data/39911/...
4,gap inc,10-Q,2023-05-26,https://www.sec.gov/Archives/edgar/data/39911/...
...,...,...,...,...
10,gap inc,10-K,2014-03-24,https://www.sec.gov/Archives/edgar/data/39911/...
11,gap inc,10-K,2013-03-26,https://www.sec.gov/Archives/edgar/data/39911/...
12,gap inc,10-K,2012-03-26,https://www.sec.gov/Archives/edgar/data/39911/...
13,gap inc,10-K,2011-03-28,https://www.sec.gov/Archives/edgar/data/39911/...


In [62]:
# Export the final DataFrame to a CSV file
df_final.to_csv('gap_filings_10k_10q.csv', index=False)


Chat GPT links:
- https://chatgpt.com/share/2a03b8be-dcf3-46af-8076-cbe79433c496
- https://chatgpt.com/share/eb826395-a910-46d9-940d-c3f803a1d6e8